# Item Response Theory with Expectation Maximization Optimization (EMIRT)

This notebook will show you how to train and use the EMIRT.
First, we will show how to get the data (here we use a0910 as the dataset).
Then we will show how to train a EMIRT and perform the parameters persistence.
At last, we will show how to load the parameters from the file and evaluate on the test dataset.

The script version could be found in [IRT.py](IRT.py)

In [ ]:
# Load the data from files
import pandas as pd

train_data = pd.read_csv("../../../data/a0910/train.csv")
valid_data = pd.read_csv("../../../data/a0910/valid.csv")
test_data = pd.read_csv("../../../data/a0910/test.csv")

train_data.head(5)

In [3]:
len(train_data), len(valid_data), len(test_data)

(186049, 25606, 55760)

In [4]:
stu_num = max(max(train_data['user_id']), max(test_data['user_id']))
prob_num = max(max(train_data['item_id']), max(test_data['item_id']))
print(stu_num, prob_num)

4128 17746


In [5]:
import numpy as np
R = -1 * np.ones(shape=(stu_num, prob_num))
R[train_data['user_id']-1, train_data['item_id']-1] = train_data['score']

test_set = []
for i in range(len(test_data)):
    row = test_data.iloc[i]
    test_set.append({'user_id':int(row['user_id'])-1, 'item_id':int(row['item_id'])-1, 'score':row['score']})

## Training and Persistence

In [5]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [8]:
from EduCDM import EMIRT

cdm = EMIRT(R, stu_num, prob_num, dim=1, skip_value=-1)

cdm.train(lr=1e-3, epoch=2)
cdm.save("irt.params")

MemoryError: Unable to allocate 559. MiB for an array with shape (17746, 4128) and data type float64

## Loading and Testing

In [5]:
cdm.load("irt.params")
rmse, mae = cdm.eval(test_set)
print("RMSE, MAE are %.6f, %.6f" % (rmse, mae))

evaluating: 100%|██████████| 55760/55760 [00:00<00:00, 774400.56it/s]

RMSE, MAE are 0.453045, 0.384567


## Incremental Training

In [8]:
new_data = [{'user_id': 0, 'item_id': 2, 'score': 0.0}, {'user_id': 1, 'item_id': 1, 'score': 1.0}]
cdm.inc_train(new_data, lr=1e-3, epoch=2)

## Evaluate User's State

In [9]:
stu_rec = np.random.randint(-1, 2, size=prob_num)
dia_state = cdm.transform(stu_rec)  # shape = (stu_num, dim)
print("user's state is " + str(dia_state))

user's state is [[73.94697385]]
